# Importar bibliotecas

In [1]:
import pandas as pd
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 30

# Ler o estoque

In [3]:
patt = './Estoque/*/*Estoque*.csv'
list_files = glob(patt)
list_files

['./Estoque\\FIDC FCT CONSIGNADO II  - Estoque 14.07.25\\FIDC FCT CONSIGNADO II  - Estoque 14.07.25-Parte1.csv',
 './Estoque\\FIDC FCT CONSIGNADO II  - Estoque 14.07.25\\FIDC FCT CONSIGNADO II  - Estoque 14.07.25-Parte2.csv',
 './Estoque\\FIDC FCT CONSIGNADO II  - Estoque 14.07.25\\FIDC FCT CONSIGNADO II  - Estoque 14.07.25-Parte3.csv',
 './Estoque\\FIDC FCT CONSIGNADO II  - Estoque 14.07.25\\FIDC FCT CONSIGNADO II  - Estoque 14.07.25-Parte4.csv']

In [4]:
file = list_files[0]

In [5]:
# Colunas que contêm datas.
colunas_data = [
    'DataEmissao',
    'DataAquisicao',
    'DataVencimento',
    'DataGeracao'
]

colunas_texto = [
    'Situacao', 'PES_TIPO_PESSOA', 'CedenteCnpjCpf', 'TIT_CEDENTE_ENT_CODIGO',
    'CedenteNome', 'Cnae', 'SecaoCNAEDescricao', 'NotaPdd', 'SAC_TIPO_PESSOA',
    'SacadoCnpjCpf', 'SacadoNome', 'IdTituloVortx', 'TipoAtivo', 'NumeroBoleto',
    'NumeroTitulo', 'CampoChave', 'PagamentoParcial', 'Coobricacao',
    'CampoAdicional1', 'CampoAdicional2', 'CampoAdicional3', 'CampoAdicional4',
    'CampoAdicional5', 'IdTituloVortxOriginador', 'Registradora',
    'IdContratoRegistradora', 'IdTituloRegistradora', 'CCB', 'Convênio'
]

#*Numéricas: 
colunas_numericas = [
    'ValorAquisicao', 'ValorNominal', 'ValorPresente', 'PDDNota', 'PDDVencido',
    'PDDTotal', 'PDDEfeitoVagao', 'PercentagemEfeitoVagao'
]

# dic pra ler as de texto
dtype_texto = {col: str for col in colunas_texto}


In [6]:
%%time
dfs = []
for file in list_files:
    df_ = pd.read_csv(file, sep=';', encoding='latin1', dtype=dtype_texto, 
                      decimal=',', parse_dates=colunas_data, dayfirst=True)
    dfs.append(df_)

CPU times: total: 25.3 s
Wall time: 26.2 s


In [7]:
# for file in list_files:
#     with open(file, 'r', encoding='latin1') as arquivo:
#         num_linhas = sum(1 for linha in arquivo)
#     print(file, num_linhas)

In [8]:
%%time
df_final = pd.concat(dfs)

CPU times: total: 11.6 s
Wall time: 12.1 s


In [9]:
# criar colunas auxiliares
df_final['_ValorLiquido'] = df_final['ValorPresente'] - df_final['PDDTotal']
df_final['_ValorVencido'] = (df_final['DataVencimento'] <= df_final['DataGeracao']).astype('int') * df_final['ValorPresente']

# sacados com muitos contratos
sacado_contratos = df_final.groupby('SacadoNome')['CCB'].nunique()
k = 3 # numero alto
mask = sacado_contratos[sacado_contratos >= k]
sacado_contratos_alto = sacado_contratos.index[mask]
df_final['_MuitosContratos'] = df_final['SacadoNome'].isin(sacado_contratos_alto).astype('str')

# sacados com muitos entes
sacados_entes = df_final.groupby('SacadoCnpjCpf')['Convênio'].nunique()
k2 = 3
mask = sacados_entes >= k2
sacados_entes_alto = sacados_entes.index[mask]
df_final['_MuitosEntes'] = df_final['SacadoCnpjCpf'].isin(sacados_entes_alto).astype('str')

# Criar coluna para sacado BMP
mask_bmp = df_final['SacadoCnpjCpf'] == '34.337.707/0001-00'
df_final['_SacadoBMP'] = mask_bmp

In [10]:
# Mapear entes
df_map_entes = pd.read_excel('MAP_ENTES.xlsx')

map_nivel = dict(zip(df_map_entes['NOME'], df_map_entes['_NIVEL']))
map_prev = dict(zip(df_map_entes['NOME'], df_map_entes['_PREV']))
map_generic = dict(zip(df_map_entes['NOME'], df_map_entes['_GENERICO']))

df_final['_NIVEL'] = df_final['Convênio'].map(map_nivel)
df_final['_PREV'] = df_final['Convênio'].map(map_prev)
df_final['_GENERICO'] = df_final['Convênio'].map(map_generic)

In [11]:
df_final[df_final['_SacadoBMP']]['ValorNominal'].sum()

np.float64(5332.350000000001)

In [12]:
df_final.memory_usage(deep=True).sum()/1024**2

np.float64(3125.946822166443)

In [13]:
# Tamanho do estoque
print(f'{df_final["ValorPresente"].sum():_.2f}')

199_232_147.56


In [14]:
# # Função para gerar DataFrame com informações das colunas
# def get_column_info(df):
#     # Criar lista com informações das colunas
#     info_data = [
#         {
#             'Coluna': col,
#             '# Dados não nulos': df[col].notnull().sum(),
#             'Tipo de Dado': str(df[col].dtype)
#         }
#         for col in df.columns
#     ]
    
#     # Criar DataFrame com as informações
#     df_info = pd.DataFrame(info_data, index=np.arange(1,len(df.columns)+1))
    
#     dict_map = {'object': 'Texto',
#             'datetime64[ns]': 'Data',
#             'float64': 'Número'}

#     df_info['Tipo de Dado'] = df_info['Tipo de Dado'].map(dict_map)
    
#     return df_info

# get_column_info(df_final).to_excel('resumo_dataframe.xlsx')

In [15]:
# color1 = np.array([255, 255, 255]) / 255
# color2 = np.array([118, 198, 197]) / 255
# color3 = np.array([14, 93, 95]) / 255
# color4 = np.array([22, 63, 63]) / 255
# color5 = np.array([49, 49, 49]) / 255

# Analisar com `value_counts()`
### Comentários / dúvidas
1. [Situacao] - diferença entre sem cobrança e aditado
1. [SAC_TIPO_PESSOA] - tipo J = jurídico?? tem isso?
1. [SacadoCnpjCpf] - por que tem CNPJ?
1. [SacadoNome] 'BMP SOCIEDADE DE CREDITO DIRETO S.A'
1. [TipoAtivo] - CCB e Contrato. Por que tem contrato?
1. [DataGeracao] - data de referência ou data de processamento?
1. [SacadoCnpjCpf] - tem sacado com 1040 linhas (!)
1. [Convênio] - sacados com muitos convênios (3)
1. Sacado tipo J está com PDD altíssima
1. Por que tem parcelas com valor de aquisicação maior do que valor presente?

In [16]:
df_final2 = df_final[~df_final['Situacao'].isna()].copy()

for col in df_final2.columns:
    if not df_final2[col].dtype == df_final2['Situacao'].dtype:
        continue
    display(df_final2[[col]].value_counts(dropna=False).to_frame().reset_index())
    print('*'*80)
    

,Situacao,count
0,Sem cobranÃ§a,1471376
1,Aditado,50939


********************************************************************************


,PES_TIPO_PESSOA,count
0,J,1522315


********************************************************************************


,CedenteCnpjCpf,count
0,34.337.707/0001-00,1519907
1,92.874.270/0001-40,2408


...

********************************************************************************


,_MuitosContratos,count
0,False,1520460
1,True,1855


********************************************************************************


,_MuitosEntes,count
0,False,1522122
1,True,193


********************************************************************************


# Verificando as 117.984 linhas com dados nulos
Pela análise abaixo, não tem influência no cálculo

In [17]:
def contar_linhas_em_branco(arquivo, encoding='latin_1'):
    try:
        with open(arquivo, 'r', encoding=encoding) as file:
            linhas = file.readlines()
            print(linhas[-1].strip(';'))
            linhas_em_branco = sum(1 for linha in linhas if linha.strip(';\n ') == '')
            return linhas_em_branco
    except FileNotFoundError:
        print(f"Erro: O arquivo '{arquivo}' não foi encontrado.")
    except Exception as e:
        print(f"Erro ao ler o arquivo: {e}")
        
contar_linhas_em_branco(list_files[3])

117984

In [18]:
df_final[df_final['Situacao'].isna()].apply(lambda x: (min(x), max(x)))

,Situacao,PES_TIPO_PESSOA,CedenteCnpjCpf,TIT_CEDENTE_ENT_CODIGO,CedenteNome,Cnae,SecaoCNAEDescricao,NotaPdd,SAC_TIPO_PESSOA,SacadoCnpjCpf,SacadoNome,IdTituloVortx,TipoAtivo,DataEmissao,DataAquisicao,DataVencimento,NumeroBoleto,NumeroTitulo,CampoChave,ValorAquisicao,ValorNominal,ValorPresente,PDDNota,PDDVencido,PagamentoParcial,Coobricacao,DataGeracao,PDDTotal,CampoAdicional1,CampoAdicional2,CampoAdicional3,CampoAdicional4,CampoAdicional5,PDDEfeitoVagao,PercentagemEfeitoVagao,IdTituloVortxOriginador,Registradora,IdContratoRegistradora,IdTituloRegistradora,CCB,Convênio,_ValorLiquido,_ValorVencido,_MuitosContratos,_MuitosEntes,_SacadoBMP
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False


In [19]:
v1_ = df_final['ValorPresente'].sum() 
v2_ = df_final[~df_final['Situacao'].isna()]['ValorPresente'].sum()
np.isclose(v1_, v2_)

np.True_

In [20]:
del df_final

# Sacado == 'J'
aqui não tem problema

In [21]:
df_sacado_J = df_final2[df_final2['SAC_TIPO_PESSOA'] == 'J'].sample(5)

In [22]:
df_sacado_J['SacadoCnpjCpf'].map(len).unique()

array([14])

# Sacado com CNPJ

In [23]:
set(df_final2['SacadoCnpjCpf'].apply(len))

{14, 18}

In [24]:
df_final2_cnpj = df_final2[df_final2['SacadoCnpjCpf'].map(len)==18]
df_final2_cnpj['SacadoNome'].unique()

array(['BMP SOCIEDADE DE CREDITO DIRETO S.A'], dtype=object)

In [25]:
# for col in df_final2_cnpj.columns:
#     if not df_final2_cnpj[col].dtype == df_final2_cnpj['Situacao'].dtype:
#         continue
#     print(df_final2_cnpj[col].value_counts(dropna=False))
#     print('*'*80)

In [26]:
df_final2_cnpj.describe(include=[np.number])

,ValorAquisicao,ValorNominal,ValorPresente,PDDNota,PDDVencido,PDDTotal,PDDEfeitoVagao,PercentagemEfeitoVagao,_ValorLiquido,_ValorVencido
count,95.000000,9.500000e+01,95.000000,95.0,95.000000,95.000000,95.000000,95.0,95.0,95.000000
mean,23.177368,5.613000e+01,27.604434,0.0,1.650542,27.604434,27.604434,1.0,0.0,4.135895
std,13.054230,1.428624e-14,15.121244,0.0,8.263526,15.121244,15.121244,0.0,0.0,14.742111
min,7.310000,5.613000e+01,8.795000,0.0,0.000000,8.795000,8.795000,1.0,0.0,0.000000
25%,12.005000,5.613000e+01,14.445900,0.0,0.000000,14.445900,14.445900,1.0,0.0,0.000000
50%,19.710000,5.613000e+01,23.720600,0.0,0.000000,23.720600,23.720600,1.0,0.0,0.000000
75%,32.350000,5.613000e+01,38.939150,0.0,0.000000,38.939150,38.939150,1.0,0.0,0.000000
max,53.070000,5.613000e+01,56.130000,0.0,56.130000,56.130000,56.130000,1.0,0.0,56.130000


In [27]:
df_final2_cnpj.select_dtypes(include=[np.number]).sum()

ValorAquisicao            2201.8500
ValorNominal              5332.3500
ValorPresente             2622.4212
PDDNota                      0.0000
PDDVencido                 156.8015
PDDTotal                  2622.4212
PDDEfeitoVagao            2622.4212
PercentagemEfeitoVagao      95.0000
_ValorLiquido                0.0000
_ValorVencido              392.9100
dtype: float64

In [29]:
# Exportar para excel
df_final2_cnpj.to_excel('df_final2_cnpj.xlsx')

# SacadoCnpjCpf - verificar consistência dos CPFs

In [30]:
def validar_cpf(cpf):
    # Remove caracteres não numéricos
    cpf = ''.join(filter(str.isdigit, str(cpf)))
    
    # Verifica se tem 11 dígitos
    if len(cpf) != 11:
        return False
    
    # Verifica se todos os dígitos são iguais (ex: 111.111.111-11)
    if cpf == cpf[0] * 11:
        return False
    
    # Cálculo do primeiro dígito verificador
    soma = 0
    for i in range(9):
        soma += int(cpf[i]) * (10 - i)
    
    resto = (soma * 10) % 11
    if resto == 10 or resto == 11:
        resto = 0
    if resto != int(cpf[9]):
        return False
    
    # Cálculo do segundo dígito verificador
    soma = 0
    for i in range(10):
        soma += int(cpf[i]) * (11 - i)
    
    resto = (soma * 10) % 11
    if resto == 10 or resto == 11:
        resto = 0
    if resto != int(cpf[10]):
        return False
    
    return True

In [31]:
df_final2['CPF_válido'] = df_final2['SacadoCnpjCpf'].apply(validar_cpf)

In [32]:
df_final2[~df_final2['CPF_válido']]['SacadoCnpjCpf'].unique()

array(['34.337.707/0001-00'], dtype=object)

In [33]:
df_final2.sample(2)

,Situacao,PES_TIPO_PESSOA,CedenteCnpjCpf,TIT_CEDENTE_ENT_CODIGO,CedenteNome,Cnae,SecaoCNAEDescricao,NotaPdd,SAC_TIPO_PESSOA,SacadoCnpjCpf,SacadoNome,IdTituloVortx,TipoAtivo,DataEmissao,DataAquisicao,DataVencimento,NumeroBoleto,NumeroTitulo,CampoChave,ValorAquisicao,ValorNominal,ValorPresente,PDDNota,PDDVencido,PagamentoParcial,Coobricacao,DataGeracao,PDDTotal,CampoAdicional1,CampoAdicional2,CampoAdicional3,CampoAdicional4,CampoAdicional5,PDDEfeitoVagao,PercentagemEfeitoVagao,IdTituloVortxOriginador,Registradora,IdContratoRegistradora,IdTituloRegistradora,CCB,Convênio,_ValorLiquido,_ValorVencido,_MuitosContratos,_MuitosEntes,_SacadoBMP,CPF_válido
20015,Sem cobranÃ§a,J,34.337.707/0001-00,318853,BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,6499999,OUTRAS ATIVIDADES DE SERVIÃOS FINANCEIROS NÃ...,AA,F,398.991.104-00,JOSIVAL C DE LIRA,69223887,CCB,2024-12-16,2024-12-17,2026-11-20,NaN,45743179022,45743179022,180.87,337.68,217.1116,0.0,0.0,NAO,NAO,2025-07-14,0.0000,NaN,NaN,NaN,NaN,NaN,0.0000,0.0,0,NaN,NaN,NaN,45743179,GOV. ALAGOAS,217.1116,0.0,False,False,False,True
110467,Sem cobranÃ§a,J,34.337.707/0001-00,318853,BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,6499999,OUTRAS ATIVIDADES DE SERVIÃOS FINANCEIROS NÃ...,AA,J,036.012.841-62,LETICIA MACHADO LUSTOSA RODRIGUES REGO,39450022,CCB,2024-04-30,2024-05-03,2028-07-15,NaN,3516996350,97416460,230.38,734.00,320.3946,0.0,0.0,NAO,NAO,2025-07-14,320.3946,NaN,NaN,NaN,NaN,NaN,320.3946,1.0,39448706,NaN,NaN,NaN,35169963,GOV. GOIAS,0.0000,0.0,False,False,False,True


# Calcular % PDD em função de variáveis categóricas

In [34]:
cat_cols = ['Situacao', 'CedenteNome', 'SAC_TIPO_PESSOA', 'PagamentoParcial', 'TipoAtivo',
            '_MuitosContratos', '_MuitosEntes', '_SacadoBMP', 'Convênio']

pdd_ref = (1- df_final2['_ValorLiquido'].sum() / df_final2['ValorPresente'].sum()) * 100
print(f'{pdd_ref:_.2f}')

for col in cat_cols:
    aux_ = df_final2.groupby(col)[['_ValorLiquido', 'ValorPresente']].sum()/1e6
    aux_['%PDD'] = (1- aux_['_ValorLiquido'] / aux_['ValorPresente']) * 100
    
    if col == 'Convênio':
        aux_ = aux_.sort_values('ValorPresente', ascending=False)
    
    display(aux_[:20])
    

19.97


,_ValorLiquido,ValorPresente,%PDD
Situacao,,,
Aditado,7.851224,8.081707,2.851911
Sem cobranÃ§a,151.600857,191.150441,20.690292


,_ValorLiquido,ValorPresente,%PDD
CedenteNome,,,
BANCO DIGIMAIS S.A.,0.246985,0.297263,16.913659
BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,159.205096,198.934885,19.971253


,_ValorLiquido,ValorPresente,%PDD
SAC_TIPO_PESSOA,,,
F,130.019122,150.625730,13.680669
J,29.432959,48.606418,39.446353


,_ValorLiquido,ValorPresente,%PDD
PagamentoParcial,,,
NAO,159.39059,198.993242,19.901506
SIM,0.06149,0.238906,74.261624


,_ValorLiquido,ValorPresente,%PDD
TipoAtivo,,,
CCB,150.961799,190.277251,20.662192
CT - Contrato,8.490281,8.954897,5.188396


,_ValorLiquido,ValorPresente,%PDD
_MuitosContratos,,,
False,159.179279,198.940162,19.986353
True,0.272802,0.291985,6.570138


,_ValorLiquido,ValorPresente,%PDD
_MuitosEntes,,,
False,159.436245,199.208493,19.965137
True,0.015835,0.023654,33.054812


,_ValorLiquido,ValorPresente,%PDD
_SacadoBMP,,,
False,159.452081,199.229525,19.965637
True,0.000000,0.002622,100.000000


,_ValorLiquido,ValorPresente,%PDD
Convênio,,,
GOV. MARANHAO,41.484712,46.568145,10.916117
GOV. ALAGOAS,32.999110,34.786090,5.137055
GOV. GOIAS,13.734775,17.185891,20.081099
PREF. BARREIRAS,9.304598,9.363838,0.632651
PREF. CABO FRIO,2.354433,6.814405,65.449174
GOV. RIO GRANDE DO NORTE,6.043314,6.765837,10.678985
PREF. CAMPOS DOS GOYTACAZES,4.330700,6.313879,31.409832
PREF. BALSAS,5.092831,5.119877,0.528237
GOV. TOCANTINS,2.902283,5.105860,43.157807


# Repetir o item anterior para vencidos

In [35]:
venc_ref = (df_final2['_ValorVencido'].sum() / df_final2['ValorPresente'].sum()) * 100
print(f'{venc_ref:_.2f}')

for col in cat_cols:
    aux_ = df_final2.groupby(col)[['_ValorVencido', 'ValorPresente']].sum()/1e6
    aux_['%Vencido'] = (aux_['_ValorVencido'] / aux_['ValorPresente']) * 100
    
    if col == 'Convênio':
        aux_ = aux_.sort_values('ValorPresente', ascending=False)
  
    display(aux_)
    

5.12


,_ValorVencido,ValorPresente,%Vencido
Situacao,,,
Aditado,0.038819,8.081707,0.480327
Sem cobranÃ§a,10.165953,191.150441,5.318299


,_ValorVencido,ValorPresente,%Vencido
CedenteNome,,,
BANCO DIGIMAIS S.A.,0.019911,0.297263,6.698064
BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,10.184861,198.934885,5.119696


,_ValorVencido,ValorPresente,%Vencido
SAC_TIPO_PESSOA,,,
F,6.282185,150.625730,4.170725
J,3.922586,48.606418,8.070099


,_ValorVencido,ValorPresente,%Vencido
PagamentoParcial,,,
NAO,9.967889,198.993242,5.009160
SIM,0.236883,0.238906,99.153147


,_ValorVencido,ValorPresente,%Vencido
TipoAtivo,,,
CCB,9.932005,190.277251,5.219754
CT - Contrato,0.272766,8.954897,3.046001


,_ValorVencido,ValorPresente,%Vencido
_MuitosContratos,,,
False,10.197323,198.940162,5.125824
True,0.007448,0.291985,2.550866


,_ValorVencido,ValorPresente,%Vencido
_MuitosEntes,,,
False,10.202956,199.208493,5.121748
True,0.001815,0.023654,7.672974


,_ValorVencido,ValorPresente,%Vencido
_SacadoBMP,,,
False,10.204379,199.229525,5.121921
True,0.000393,0.002622,14.982719


,_ValorVencido,ValorPresente,%Vencido
Convênio,,,
GOV. MARANHAO,0.909802,46.568145,1.953700
GOV. ALAGOAS,1.368971,34.786090,3.935397
GOV. GOIAS,1.110039,17.185891,6.459012
PREF. BARREIRAS,0.018141,9.363838,0.193732
PREF. CABO FRIO,0.873612,6.814405,12.820073
...,...,...,...
PREV. PORTO CALVO,0.000450,0.002199,20.463727
PREF. CAMPINA GRANDE,0.000000,0.001643,0.000000
PREF. LEOPOLDO DE BULHOES,0.000131,0.001613,8.118001


# Sacados em mais de um ente?

In [36]:
# SacadoCnpjCpf , SacadoNome
sacados_entes = df_final2.groupby('SacadoCnpjCpf')['Convênio'].agg([pd.Series.nunique, pd.unique])
sacados_entes = sacados_entes.sort_values('nunique', ascending=False)[:35]
sacados_entes['unique'] = sacados_entes['unique'].apply(lambda x: ' | '.join(x))
sacados_entes

,nunique,unique
SacadoCnpjCpf,,
323.026.574-20,3,GOV. GOIAS | GOV. SÃO PAULO | GOV. RIO GRANDE ...
048.692.204-90,3,GOV. GOIAS | PREF. TUTOIA | GOV. RIO GRANDE DO...
109.273.037-01,2,PREF. CABO FRIO | PREF. SAQUAREMA
114.742.747-06,2,PREF. CABO FRIO | PREF. SAQUAREMA
292.798.363-15,2,GOV. MARANHAO | PREF. BALSAS
...,...,...
996.539.907-72,1,PREF. SAQUAREMA
996.568.337-91,1,PREF. CABO FRIO
996.699.942-68,1,PREF. ÓBIDOS


In [37]:
sacados_entes.to_excel('sacados_entes.xlsx')

In [38]:
df_final2.columns

Index(['Situacao', 'PES_TIPO_PESSOA', 'CedenteCnpjCpf',
       'TIT_CEDENTE_ENT_CODIGO', 'CedenteNome', 'Cnae', 'SecaoCNAEDescricao',
       'NotaPdd', 'SAC_TIPO_PESSOA', 'SacadoCnpjCpf', 'SacadoNome',
       'IdTituloVortx', 'TipoAtivo', 'DataEmissao', 'DataAquisicao',
       'DataVencimento', 'NumeroBoleto', 'NumeroTitulo', 'CampoChave',
       'ValorAquisicao', 'ValorNominal', 'ValorPresente', 'PDDNota',
       'PDDVencido', 'PagamentoParcial', 'Coobricacao', 'DataGeracao',
       'PDDTotal', 'CampoAdicional1', 'CampoAdicional2', 'CampoAdicional3',
       'CampoAdicional4', 'CampoAdicional5', 'PDDEfeitoVagao',
       'PercentagemEfeitoVagao', 'IdTituloVortxOriginador', 'Registradora',
       'IdContratoRegistradora', 'IdTituloRegistradora', 'CCB', 'Convênio',
       '_ValorLiquido', '_ValorVencido', '_MuitosContratos', '_MuitosEntes',
       '_SacadoBMP', 'CPF_válido'],
      dtype='object')

In [39]:
df_final2.sample(2)

,Situacao,PES_TIPO_PESSOA,CedenteCnpjCpf,TIT_CEDENTE_ENT_CODIGO,CedenteNome,Cnae,SecaoCNAEDescricao,NotaPdd,SAC_TIPO_PESSOA,SacadoCnpjCpf,SacadoNome,IdTituloVortx,TipoAtivo,DataEmissao,DataAquisicao,DataVencimento,NumeroBoleto,NumeroTitulo,CampoChave,ValorAquisicao,ValorNominal,ValorPresente,PDDNota,PDDVencido,PagamentoParcial,Coobricacao,DataGeracao,PDDTotal,CampoAdicional1,CampoAdicional2,CampoAdicional3,CampoAdicional4,CampoAdicional5,PDDEfeitoVagao,PercentagemEfeitoVagao,IdTituloVortxOriginador,Registradora,IdContratoRegistradora,IdTituloRegistradora,CCB,Convênio,_ValorLiquido,_ValorVencido,_MuitosContratos,_MuitosEntes,_SacadoBMP,CPF_válido
545146,Sem cobranÃ§a,J,34.337.707/0001-00,318853,BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,6499999,OUTRAS ATIVIDADES DE SERVIÃOS FINANCEIROS NÃ...,AA,F,003.488.395-96,SUZANA MARCIA SOUZA DOS SANTOS,99194683,CCB,2025-06-10,2025-06-11,2032-11-15,NaN,56865375089,56865375089,44.45,318.32,45.4966,0.0,0.0,NAO,NAO,2025-07-14,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,NaN,NaN,NaN,56865375,PREF. CASA NOVA,45.4966,0.0,False,False,False,True
458575,Sem cobranÃ§a,J,34.337.707/0001-00,318853,BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,6499999,OUTRAS ATIVIDADES DE SERVIÃOS FINANCEIROS NÃ...,AA,F,811.234.692-53,LEANDRO ALVES DE LIRA,66892693,CCB,2024-11-25,2024-11-26,2028-09-15,NaN,44546346045,182074596,12.72,49.98,15.9099,0.0,0.0,NAO,NAO,2025-07-14,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,NaN,NaN,NaN,44546346,PREF. BOA VISTA,15.9099,0.0,False,False,False,True


# Verificação de datas

In [40]:
(df_final2['DataEmissao'] > df_final2['DataAquisicao']).sum() # ok

np.int64(0)

In [41]:
(df_final2['DataAquisicao'] > df_final2['DataVencimento']).sum() # ok

np.int64(0)

In [42]:
df_final2['DataAquisicao'].min(), df_final2['DataAquisicao'].max() 

(Timestamp('2024-01-18 00:00:00'), Timestamp('2025-07-14 00:00:00'))

# Verificação dos valores

In [43]:
(df_final2['ValorAquisicao'] > df_final2['ValorNominal']).sum() # ok

np.int64(3696)

In [44]:
(df_final2['ValorAquisicao'] > df_final2['ValorPresente']).sum() # ok

np.int64(3696)

In [45]:
(df_final2['ValorPresente'] > df_final2['ValorNominal']).sum() # ok

np.int64(0)

In [46]:
mask = df_final2['ValorAquisicao'] > df_final2['ValorNominal']
display(df_final2[mask]['PagamentoParcial'].value_counts())
df_final2[mask].sample(3)

PagamentoParcial
SIM    3696
Name: count, dtype: int64

,Situacao,PES_TIPO_PESSOA,CedenteCnpjCpf,TIT_CEDENTE_ENT_CODIGO,CedenteNome,Cnae,SecaoCNAEDescricao,NotaPdd,SAC_TIPO_PESSOA,SacadoCnpjCpf,SacadoNome,IdTituloVortx,TipoAtivo,DataEmissao,DataAquisicao,DataVencimento,NumeroBoleto,NumeroTitulo,CampoChave,ValorAquisicao,ValorNominal,ValorPresente,PDDNota,PDDVencido,PagamentoParcial,Coobricacao,DataGeracao,PDDTotal,CampoAdicional1,CampoAdicional2,CampoAdicional3,CampoAdicional4,CampoAdicional5,PDDEfeitoVagao,PercentagemEfeitoVagao,IdTituloVortxOriginador,Registradora,IdContratoRegistradora,IdTituloRegistradora,CCB,Convênio,_ValorLiquido,_ValorVencido,_MuitosContratos,_MuitosEntes,_SacadoBMP,CPF_válido
242050,Sem cobranÃ§a,J,34.337.707/0001-00,318853,BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,6499999,OUTRAS ATIVIDADES DE SERVIÃOS FINANCEIROS NÃ...,AA,J,136.968.153-49,JOSE RIBAMAR SERRA PIRES,38073713,CCB,2024-04-23,2024-04-25,2025-06-03,NaN,3484655813,85590840,184.05,3.01,3.01,0.0,0.0,SIM,NAO,2025-07-14,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,NaN,NaN,NaN,34846558,GOV. MARANHAO,3.01,3.01,False,False,False,True
203364,Sem cobranÃ§a,J,34.337.707/0001-00,318853,BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,6499999,OUTRAS ATIVIDADES DE SERVIÃOS FINANCEIROS NÃ...,AA,J,216.385.513-91,EDNA MARIA MENDONCA DE ASSIS,37331408,CCB,2024-04-08,2024-04-09,2025-06-03,NaN,3426891313,61348513,69.10,3.01,3.01,0.0,0.0,SIM,NAO,2025-07-14,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,NaN,NaN,NaN,34268913,GOV. MARANHAO,3.01,3.01,False,False,False,True
227970,Sem cobranÃ§a,J,34.337.707/0001-00,318853,BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,6499999,OUTRAS ATIVIDADES DE SERVIÃOS FINANCEIROS NÃ...,AA,F,027.470.103-09,JADERSON LIMA RIBEIRO,82844824,CCB,2025-02-27,2025-02-27,2025-06-25,NaN,49905062002,49905062002,137.65,3.01,3.01,0.0,0.0,SIM,NAO,2025-07-14,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,NaN,NaN,NaN,49905062,GOV. MARANHAO,3.01,3.01,False,False,False,True


In [47]:
df_final2.columns

Index(['Situacao', 'PES_TIPO_PESSOA', 'CedenteCnpjCpf',
       'TIT_CEDENTE_ENT_CODIGO', 'CedenteNome', 'Cnae', 'SecaoCNAEDescricao',
       'NotaPdd', 'SAC_TIPO_PESSOA', 'SacadoCnpjCpf', 'SacadoNome',
       'IdTituloVortx', 'TipoAtivo', 'DataEmissao', 'DataAquisicao',
       'DataVencimento', 'NumeroBoleto', 'NumeroTitulo', 'CampoChave',
       'ValorAquisicao', 'ValorNominal', 'ValorPresente', 'PDDNota',
       'PDDVencido', 'PagamentoParcial', 'Coobricacao', 'DataGeracao',
       'PDDTotal', 'CampoAdicional1', 'CampoAdicional2', 'CampoAdicional3',
       'CampoAdicional4', 'CampoAdicional5', 'PDDEfeitoVagao',
       'PercentagemEfeitoVagao', 'IdTituloVortxOriginador', 'Registradora',
       'IdContratoRegistradora', 'IdTituloRegistradora', 'CCB', 'Convênio',
       '_ValorLiquido', '_ValorVencido', '_MuitosContratos', '_MuitosEntes',
       '_SacadoBMP', 'CPF_válido'],
      dtype='object')

# Valor por sacado

In [48]:
df_final2.groupby('SacadoCnpjCpf')['ValorPresente'].sum().max() #sort_values(ascending=False)

np.float64(149332.211)

In [53]:
ll = df_final2['Convênio'].unique().tolist()
for i in ll:
    print(i)

ASSEMBLEIA. MATO GROSSO
GOV. ALAGOAS
GOV. AMAZONAS
GOV. ESPIRITO SANTO
GOV. GOIAS
GOV. MARANHAO
GOV. PARAIBA
GOV. PARANA
GOV. PIAUI
GOV. RIO DE JANEIRO
GOV. RIO GRANDE DO NORTE
GOV. SANTA CATARINA
GOV. SÃO PAULO
GOV. TOCANTINS
IMA CAMPINAS
PREF. ABAETETUBA
PREF. ÁGUAS BELAS
PREF. ALTINHO
PREF. ARAPONGAS
PREF. BALNEÁRIO CAMBORIÚ
PREF. BALSAS
PREF. BARREIRAS
PREF. BARREIRINHAS
PREF. BELO HORIZONTE
PREF. BELO JARDIM
PREF. BOA VISTA
PREF. CABO FRIO
PREF. CACHOEIRA GRANDE
PREF. CAJAMAR
PREF. CAMPOS DO JORDÃO
PREF. CAMPOS DOS GOYTACAZES
PREF. CANINDE
PREF. CARIACICA
PREF. CASA NOVA
PREF. CASTANHAL
PREF. CAXIAS
PREF. CEARÁ MIRIM
PREF. COTIA
PREF. CURITIBA
PREF. DRACENA
PREF. DUQUE DE CAXIAS
PREF. FEIRA DE SANTANA
PREF. FLORIANÓPOLIS
PREF. FOZ DO IGUAÇU
PREF. GOVERNADOR NUNES FREIRE
PREF. HORTOLÂNDIA
PREF. IATI
PREF. ICATU
PREF. IRARA
PREF. ITAPECERICA DA SERRA
PREF. ITAPETININGA
PREF. ITAPIPOCA
PREF. ITUPEVA
PREF. JUAZEIRO DO NORTE
PREF. JURUTI
PREF. LOUVEIRA
PREF. MACAÍBA
PREF. MARINGA
PREF.